In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración visual
sns.set_theme(style="whitegrid")

# 1. CONEXIÓN Y CARGA
# -------------------------------------------------------------------------
DB_PATH = Path("../data/processed/enigh_unificada.db")
conn = sqlite3.connect(DB_PATH)

# Cargamos todo (o filtra las columnas si pesa mucho)
df = pd.read_sql("SELECT * FROM tabla_analitica_final", conn)
conn.close()

# 2. CREACIÓN DE VARIABLES (Traducción de tu 'gen' / 'replace')
# -------------------------------------------------------------------------
# Variable dependiente binaria
df['asis_esc_bin'] = df['asis_esc'].apply(lambda x: 1 if x == 1 else 0)

# Ingreso en miles
df['ingreso_pc_miles'] = df['ingreso_total_hogar_per_capita_sin_jovenes'] / 1000

# Etiquetas
df['sexo_lbl'] = df['sexo'].map({1: 'Hombre', 2: 'Mujer'})
df['internet_lbl'] = df['tiene_internet'].map({1: 'Sí', 0: 'No'})

# 3. FUNCIÓN DE MEDIA PONDERADA (El reemplazo de 'svy: mean')
# -------------------------------------------------------------------------
def weighted_mean(data, value_col, weight_col):
    try:
        return np.average(data[value_col], weights=data[weight_col])
    except ZeroDivisionError:
        return 0

# 4. ANÁLISIS DESCRIPTIVO (Ejemplo Año 2018)
# -------------------------------------------------------------------------
print("--- ANÁLISIS DESCRIPTIVO 2018 ---")
df_18 = df[df['anio'] == 2018].copy()

# A) Promedio General de Asistencia (Ponderado)
promedio_asistencia = weighted_mean(df_18, 'asis_esc_bin', 'factor')
print(f"Tasa de Asistencia Global (2018): {promedio_asistencia:.2%}")

# B) Asistencia por Edad (El equivalente a 'over(edad)')
# Agrupamos, calculamos media ponderada para cada edad
res_edad = df_18.groupby('edad').apply(
    lambda x: weighted_mean(x, 'asis_esc_bin', 'factor')
).reset_index(name='tasa_asistencia')

# Gráfica rápida
plt.figure(figsize=(10, 5))
sns.lineplot(data=res_edad, x='edad', y='tasa_asistencia', marker='o')
plt.title("Asistencia Escolar por Edad (2018)")
plt.ylabel("Tasa Ponderada")
plt.axvline(15, color='r', linestyle='--')
plt.axvline(18, color='r', linestyle='--')
plt.show()

# C) Asistencia por Sexo (Tabla cruzada ponderada)
# En Python hacemos esto manual o con una función auxiliar
res_sexo = df_18.groupby('sexo_lbl').apply(
    lambda x: weighted_mean(x, 'asis_esc_bin', 'factor')
).reset_index(name='tasa_asistencia')

print("\nAsistencia por Sexo (2018):")
print(res_sexo)

# D) Desglose por Región
res_region = df_18.groupby('region').apply(
    lambda x: weighted_mean(x, 'asis_esc_bin', 'factor')
).reset_index(name='tasa_asistencia').sort_values('tasa_asistencia')

plt.figure(figsize=(10, 5))
sns.barplot(data=res_region, x='tasa_asistencia', y='region', palette='viridis')
plt.title("Asistencia por Región (2018)")
plt.show()

KeyError: 'ingreso_total_hogar_per_capita_sin_jovenes'